In [0]:
'''Training and saving neural network to classify Iris'''
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os,csv,math,sys, joblib
import numpy as np
import sklearn.model_selection, sklearn.preprocessing
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

class neural_network(nn.Module):
    def __init__(self, num_input, num_hidden,num_output):
        super(neural_network, self).__init__()
        self.hidden = nn.Linear(num_input,num_hidden)
        self.out = nn.Linear(num_hidden,num_output)

    def forward(self, x):
        x = F.relu(self.hidden(x))
        return self.out(x)

# Make results reproducible
seed = 1
np.random.seed(seed)
torch.manual_seed(seed)

# Loading the dataset
dataset = pd.read_csv('Iris_Dataset.csv')
dataset = pd.get_dummies(dataset, columns=['Species']) # One Hot Encoding
values = list(dataset.columns.values)

y = dataset[values[-3:]]
y = np.array(y, dtype='float32')
X = dataset[values[:-3]]
X = np.array(X, dtype='float32')

# Shuffle Data
indices = np.random.choice(len(X), len(X), replace=False)
X_values = X[indices]
y_values = y[indices]

# Creating a Train and a Test Dataset
test_size = 30
X_test = X_values[-test_size:]
X_train = X_values[:-test_size]

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_test = y_values[-test_size:]


y_train = y_values[:-test_size]

# Interval / Epochs
interval = 50
epoch = 500

loss_func = nn.CrossEntropyLoss(weight=None, size_average=True, ignore_index=-100, reduce=True)

# Input neurons : 4
# Hidden neurons : 8
# Output neurons : 3
hidden_layer_nodes = 8
model = neural_network(4,hidden_layer_nodes,3)

optimizer = optim.Adam([
                {'params': model.parameters()},
            ], lr = 0.0003, weight_decay=1e-4)


for epoch in range(0,epoch): 
    loss_val = 0
    for input_data in enumerate(X_train):
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        input_neurons_data = autograd.Variable(torch.FloatTensor(input_data[1]))


        one_hot_target = y_train[input_data[0]]
        target_class_index = int(np.where(one_hot_target == 1.0)[0])

        target_class = autograd.Variable(torch.LongTensor([target_class_index]))
        output = model(input_neurons_data)

        loss = loss_func(output.view(1,-1),target_class)
        loss_val += loss

        loss.backward()
        optimizer.step()

    if epoch%interval == 0:
        print (loss_val.item()/len(X_train))

torch.save(model, "iris_double_model_8")

error = 0.0
for i in range(0,len(X_test)):
    predicted = np.argmax(np.array(model(autograd.Variable(torch.FloatTensor(X_test[i]))).data))
    actual = int(np.where(y_test[i] == 1.0)[0])
    if predicted != actual:
        error += 1.0

print (error/len(X_test))

error = 0.0
for i in range(0,len(X_train)):
    predicted = np.argmax(np.array(model(autograd.Variable(torch.FloatTensor(X_train[i]))).data))
    actual = int(np.where(y_train[i] == 1.0)[0])
    if predicted != actual:
        error += 1.0

print (error/len(X_train))

/usr/local/lib/python3.6/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


1.108814493815104
0.48065096537272134
0.3008557319641113
0.20329108238220214
0.1418273131052653
0.10645585060119629
0.08524239857991536
0.07196439107259114
0.06318517923355102
0.057092559337615964
0.1
0.008333333333333333


/usr/local/lib/python3.6/site-packages/torch/serialization.py:250: UserWarning: Couldn't retrieve source code for container of type neural_network. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
